In [11]:
!pip install -q requests beautifulsoup4 langchain
!pip install -q sentence-transformers faiss-cpu
%pip install torch
!pip install ctransformers streamlit
!pip install huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## 1. Scraping & Knowledge Base Building

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time

visited = set()

def scrape_site(start_url, allowed_domain, max_pages=100):
    queue = [start_url]
    pages_scraped = 0
    scraped_data = []

    while queue and pages_scraped < max_pages:
        url = queue.pop(0)

        if url in visited:
            continue

        print(f"Scraping: {url}")
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                
                # extract text
                page_text = soup.get_text(separator=' ', strip=True)
                scraped_data.append({
                    "url": url,
                    "text": page_text
                })

                # find new links
                for link_tag in soup.find_all("a", href=True):
                    href = link_tag["href"]
                    full_url = urljoin(url, href)
                    parsed = urlparse(full_url)
                    if allowed_domain in parsed.netloc and full_url not in visited:
                        queue.append(full_url)

                pages_scraped += 1
                visited.add(url)
                
                time.sleep(0.5)  # be nice to server
        except Exception as e:
            print(f"Error scraping {url}: {e}")
    
    return scraped_data

if __name__ == "__main__":
    # contoh untuk domain uksw
    urls_to_scrape = [
        "https://www.uksw.edu/",
        "https://fti.uksw.edu/",
        "https://www.uksw.edu/akademik/fakultas"
    ]

    all_data = []
    for u in urls_to_scrape:
        data = scrape_site(u, "uksw.edu", max_pages=30)
        all_data.extend(data)
    
    # simpan ke json
    import json
    with open("uksw_scraped.json", "w", encoding="utf-8") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=2)


Scraping: https://www.uksw.edu/
Scraping: https://www.uksw.edu/#content
Scraping: https://www.uksw.edu/riset-uksw/
Scraping: https://www.uksw.edu/international-office/
Scraping: https://www.uksw.edu
Scraping: https://www.uksw.edu/sejarah/
Scraping: https://www.uksw.edu/pendiri/
Scraping: https://www.uksw.edu/lambang/
Scraping: https://www.uksw.edu/visi-misi/
Scraping: https://www.uksw.edu/gereja-pendukung/
Scraping: https://www.uksw.edu/pimpinan-universitas/
Scraping: https://www.uksw.edu/pimpinan-fakultas/
Error scraping https://www.uksw.edu/pimpinan-fakultas/: HTTPSConnectionPool(host='www.uksw.edu', port=443): Read timed out. (read timeout=10)
Scraping: https://www.uksw.edu/pimpinan-direktorat/
Scraping: https://www.uksw.edu/senat-universitas/
Scraping: https://www.uksw.edu/fakultas/
Scraping: https://www.uksw.edu/sekolah-laboratorium/
Error scraping https://www.uksw.edu/sekolah-laboratorium/: HTTPSConnectionPool(host='www.uksw.edu', port=443): Read timed out. (read timeout=10)
Scra

In [5]:
import json
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. Function membersihkan teks
def clean_text(text: str) -> str:
    # Hapus multiple whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Hapus karakter aneh (optional)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    
    return text.strip()

# 2. Function chunking teks
def chunk_text(text, chunk_size=1000, chunk_overlap=200):
    """
    Membagi teks panjang menjadi potongan (chunks)
    agar tidak terlalu panjang untuk LLM.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " "]
    )
    return splitter.split_text(text)

# 3. Load JSON hasil scraping
def preprocess_scraped_json(json_path, chunk_size=1000, chunk_overlap=200):
    with open(json_path, "r", encoding="utf-8") as f:
        scraped_data = json.load(f)
    
    documents = []
    
    for page in scraped_data:
        url = page.get("url", "")
        raw_text = page.get("text", "")
        
        # Cleaning
        cleaned_text = clean_text(raw_text)
        
        # Chunking
        chunks = chunk_text(cleaned_text, chunk_size, chunk_overlap)
        
        # Buat list document dict
        for chunk in chunks:
            documents.append({
                "text": chunk,
                "metadata": {
                    "source": url
                }
            })
    
    return documents


processed_docs = preprocess_scraped_json(
    "uksw_scraped.json",
    chunk_size=1000,
    chunk_overlap=200
)

print(f"Total chunks: {len(processed_docs)}")
print(processed_docs[0])

# Simpan hasil ke JSON
with open("uksw_chunks.json", "w", encoding="utf-8") as f:
    json.dump(processed_docs, f, ensure_ascii=False, indent=2)


Total chunks: 477
{'text': 'Home - Universitas Kristen Satya Wacana Skip to content Informasi Riset & Inovasi International Office Beranda Tentang UKSW Sejarah Pendiri Lambang Visi & Misi Sinode Gereja Pendukung Pimpinan Pimpinan Universitas Pimpinan Fakultas Pimpinan Lembaga & Direktorat Senat Universitas Pendidikan Fakultas Sekolah Laboratorium Penunjang Lembaga & Direktorat Pembangunan Berkelanjutan SIASat Flexible Learning EJournal DigiLib Repository Risat Informasi Informasi Informasi Akademik Informasi Alumni Informasi Beasiswa Ucapan Digital Fasilitas Klinik Pratama Language Training Center Pusat Studi Perpustakaan Fasilitas Lainnya SIASat Flexible Learning EJournal DigiLib Repository Risat Kehidupan Kampus Kehidupan Kampus Unggul UKSW Bangga UKSW Go Green Pencegahan dan Penanganan Kekerasan Seksual SWS Basketball Voice of SWCU Unduhan Brosur Booklet Buletin Senin Kalender Akademik Design Guideline SIASat Flexible Learning EJournal DigiLib Repository Risat SDGs Portal X Pendafta

## 2. Embeddings

In [21]:
import json

def load_chunks(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

In [22]:
docs = load_chunks("uksw_chunks.json")
print("Total chunks:", len(docs))
print("Contoh:", docs[0])

Total chunks: 477
Contoh: {'text': 'Home - Universitas Kristen Satya Wacana Skip to content Informasi Riset & Inovasi International Office Beranda Tentang UKSW Sejarah Pendiri Lambang Visi & Misi Sinode Gereja Pendukung Pimpinan Pimpinan Universitas Pimpinan Fakultas Pimpinan Lembaga & Direktorat Senat Universitas Pendidikan Fakultas Sekolah Laboratorium Penunjang Lembaga & Direktorat Pembangunan Berkelanjutan SIASat Flexible Learning EJournal DigiLib Repository Risat Informasi Informasi Informasi Akademik Informasi Alumni Informasi Beasiswa Ucapan Digital Fasilitas Klinik Pratama Language Training Center Pusat Studi Perpustakaan Fasilitas Lainnya SIASat Flexible Learning EJournal DigiLib Repository Risat Kehidupan Kampus Kehidupan Kampus Unggul UKSW Bangga UKSW Go Green Pencegahan dan Penanganan Kekerasan Seksual SWS Basketball Voice of SWCU Unduhan Brosur Booklet Buletin Senin Kalender Akademik Design Guideline SIASat Flexible Learning EJournal DigiLib Repository Risat SDGs Portal X 

In [23]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

texts = [doc["text"] for doc in docs]
vectors = embedder.encode(texts, show_progress_bar=True, batch_size=64)

# buat FAISS index
dim = vectors.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(vectors))

# mapping index ke doc
id_map = {i: docs[i] for i in range(len(docs))}

print("Index size:", index.ntotal)


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Index size: 477


## 3. Model

In [ ]:
from huggingface_hub import hf_hub_download

# Ganti tokenmu sendiri
hf_token = ""
model_id = "NousResearch/Hermes-2-Pro-Mistral-7B-GGUF"
filename = "Hermes-2-Pro-Mistral-7B.Q4_K_M.gguf"

local_path = hf_hub_download(
    repo_id=model_id,
    filename=filename,
    token=hf_token,
    local_dir="models"
)

print("Model downloaded to:", local_path)


Hermes-2-Pro-Mistral-7B.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

Model downloaded to: models/Hermes-2-Pro-Mistral-7B.Q4_K_M.gguf


In [17]:
from ctransformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_path_or_repo_id="models/Hermes-2-Pro-Mistral-7B.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=0 # ubah kalau mau pakai GPU
)



Fakultas Teknologi Informasi UKSW adalah salah satu fakultas yang berada di Universitas Kristen Satya Wacana (UKSW) Yogyakarta. Fakultas ini telah dibentuk pada tahun 1960-an untuk menyebarkan ilmu di bidang Teknologi Informasi dan Komputerisasi.

Fakultas Teknologi Informasi UKSW memiliki visi untuk mengembangkan generation yang berkompeten, berpikir kritis, dan berbudaya agama yang baik dalam bidang Teknologi Informasi dan Komputerisasi. Misi fakultas ini adalah mencapai kualitas pendidikan yang tinggi, meningkatkan keterampilan siswa, dan membentuk lulusan yang berkarya dan berwawasan agama yang baik.

Fakultas Teknologi Informasi UKSW menyediakan program studi di bidang Teknologi Informasi yang dibagi menjadi tiga jenis prodi, yaitu:




In [24]:
def search(query, index, id_map, embedder, top_k=3):
    q_emb = embedder.encode([query])
    distances, indices = index.search(q_emb, top_k)

    results = []
    for idx in indices[0]:
        results.append(id_map[idx])
    return results
